In [ ]:
import pretty_midi
import numpy as np
import itertools
import random
import glob
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn

#All the MIDIs have been converted into CSVs to be quickly read into the RNN are all stored in this folder
def getMidiCSV():
    return glob.glob("Midi-CSVs/*.csv")

#For the creation of the label array for the RNN
def get_label_vector(label):
    #Stores the correct label of the midi file by coverting the index value of at the corresponding genre to a 1
    #Is a vector for batch purposes
    label_array = np.zeros((7))
    label_dict = {'hh':0,
                 'cl':1,
                 'cn':2,
                 'ro':3,
                 'ed':4,
                 'pp':5,
                 'mt':6}
    label_array[label_dict[label]] = 1
    return label_array

def generateMidiData():
    #Finds the file names of all the MIDI CSVs and stored them into a list
    MidiCSVs = getMidiCSV()
    
    #Must have at least one midi to run program and since all are same size open first to see the size of the numpy
    sizeCheck = np.loadtxt(MidiCSVs[0], delimiter=',')
    
    #Store size to pass to midi_note for RNN
    samples = sizeCheck.shape[0]
    
    #Generates numpy arrays to hold all note matrices and corresponding labels
    midi_note = np.zeros(shape=(len(MidiCSVs),samples,128))
    midi_label = np.zeros(shape=(len(MidiCSVs),7))

    for i in range(len(MidiCSVs)):
        csv = MidiCSVs[i]
        #Loads matrix at i-th position into numpy array
        noteMatrix = np.loadtxt(csv, delimiter=',')
        
        #convert index value of midi matrix with the current notematrix
        midi_note[i] = noteMatrix

        #stores the label of the midi file which is the first two letters of each midi after the path has been removed
        abbrev = csv.split("\\")[1][:2]
        midi_label[i] = get_label_vector(abbrev)
    
    return midi_note, midi_label, samples

def train_test_split(raw_data_x, raw_data_y, decimal_split):
    #Creates a randoms set of numbers of the size data to randomize the raw data
    length = raw_data_x.shape[0]
    permutation = np.random.permutation(length)
    
    #Randomizes the data but makes sure that the x'values and the y'classifications stay correlated
    shuffled_raw_data_x = raw_data_x[permutation]
    shuffled_raw_data_y = raw_data_y[permutation]
    
    #Finds the length of the split
    idx = int(length * decimal_split)
    
    #Cuts the training, testing slices from the randomly shuffled sets
    train_x = shuffled_raw_data_x[:idx]
    test_x = shuffled_raw_data_x[idx:]
    train_y = shuffled_raw_data_y[:idx]
    test_y = shuffled_raw_data_y[idx:]
    return train_x, test_x, train_y, test_y

def RNN(x, weights, biases, timesteps, num_hidden):

    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.LSTMCell(num_hidden, forget_bias=1.0)

    #Dropout layer
    lstm_cell = rnn.DropoutWrapper(lstm_cell, output_keep_prob=0.9) #0.9
    
    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

def main():
    
    #Splits the data randomly into testing and training
    midi_note, midi_label, samples = generateMidiData()
    decimal_split = 0.85
    train_x, test_x, train_y, test_y = train_test_split(midi_note, midi_label, decimal_split)
    
    # Training Parameters
    learning_rate = 0.002 #0.002
    training_steps = 400 #400
    batch_size = train_x.shape[0]
    display_step = 10

    # Network Parameters
    num_input = 128 #instruments
    timesteps = samples # timesteps
    num_hidden = 18 #hidden layer num of features #18
    num_classes = 7 #Total amount of genres
    
    # tf Graph input
    X = tf.placeholder("float", [None, timesteps, num_input])
    Y = tf.placeholder("float", [None, num_classes])
    
    # Define weights and bias as normalized gaussian values to prevent issues from initializing as 0
    weights = {
        'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([num_classes]))
    }
    
    #Output of the RNN
    logits = RNN(X, weights, biases, timesteps, num_hidden)

    #Prediction made from the softmax output of the RNN
    prediction = tf.nn.softmax(logits)

    # Define loss and optimizer
    #Loss is the softman cross entropy as we are doing prediction and with logits as using outputs from RNN
    #Optimizer was ADAM as did best
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=Y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

    #Gradient clipping
    #Deals with the exploding gradient problem
    gradients, variables = zip(*optimizer.compute_gradients(loss_op))

    #Clips it by the maximum L2 norm
    gradients, _ = tf.clip_by_global_norm(gradients, 1.0) #1.0

    #Apply the gradients to our optimizer
    train_op = optimizer.apply_gradients(zip(gradients, variables))

    # Evaluate model by taking the highest prediction and setting to 1
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
        
    # Start training
    with tf.Session() as sess:

        # Run the initializer
        sess.run(init)

        #Runs for the defined epochs
        for step in range(1, training_steps+1):
            #Over all the data in the batch sizes
            for i in range(0, train_x.shape[0], batch_size):
                batch_x = train_x[i:i+batch_size]
                batch_y = train_y[i:i+batch_size]
                # Run optimization op (backprop)
                sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
                if step % display_step == 0 or step == 1:
                    # Calculate batch loss and accuracy
                    loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                         Y: batch_y})
                    print("Step " + str(step) + ", Minibatch Loss= " + \
                          "{:.4f}".format(loss) + ", Training Accuracy= " + \
                          "{:.3f}".format(acc))


        print("Optimization Finished!")

        # Calculate accuracy for test over batch sizes also
        for i in range(0, test_x.shape[0], batch_size):
            testing_data = test_x[i:i+batch_size]
            testing_label = test_y[i:i+batch_size]
            print("Testing Accuracy:", \
                sess.run(accuracy, feed_dict={X: testing_data, Y: testing_label}))    
        
    sess.close()
        
if __name__== "__main__":
    main()
